In [ ]:
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partaa
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partab
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partac
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partad

!cat Dataset.tar.gz.part* > Dataset.tar.gz
!rm Dataset.tar.gz.partaa
!rm Dataset.tar.gz.partab
!rm Dataset.tar.gz.partac
!rm Dataset.tar.gz.partad
# unzip the file
!tar zxf Dataset.tar.gz
!rm Dataset.tar.gz

In [ ]:
!tar zxf Dataset.tar.gz

In [ ]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed) # 设置 NumPy 的随机种子
    random.seed(seed)  # 设置 Python 内置 random 模块的随机种子
    torch.manual_seed(seed) # 设置 PyTorch 的 CPU 随机种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # 设置当前 GPU 的随机种子
        torch.cuda.manual_seed_all(seed) # 设置所有 GPU 的随机种子（多卡时）
    # 如果使用 GPU，额外设置 CUDA 的随机种子，保证 GPU 上的随机操作（如 torch.rand()）可复现。
    torch.backends.cudnn.benchmark = False
    # benchmark=False：禁止 CuDNN 自动选择最快的卷积算法（因为不同算法可能导致微小数值差异）。
    torch.backends.cudnn.deterministic = True
    # deterministic=True：强制 CuDNN 使用确定性算法，牺牲速度换取严格可复现性。

set_seed(87)
# 调用后，所有随机操作（如模型初始化、数据加载时的打乱）将基于种子 87 生成，确保多次运行结果一致。
print("Complete")

为什么需要这个类？
统一输入尺寸：神经网络需要固定长度的输入（比如128帧），但录音长短不一，必须裁剪或填充。

打乱数据：通过 DataLoader 可以随机打乱顺序，避免模型死记硬背。

说话人分类：每个片段对应一个说话人ID，模型的目标是学会区分不同人的声音。

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len
    
        # 有一本"电话簿"(mapping.json)，记录每个说话人的名字和对应的ID(比如"张三→1，李四→2")
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]
    
        # 有一张"录音清单"(metadata.json)，记录每个录音文件的位置和属于哪个说话人
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
    
        # 把所有录音文件的路径和对应的说话人ID整理成一个列表(self.data)
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
    
    # 有多少条录音
    def __len__(self):
        return len(self.data)  # 即所有语音文件的数量
 
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # 加载 Mel 频谱图
        mel = torch.load(os.path.join(self.data_dir, feat_path))
        # 长的随机剪一段短的直接整段用(可能会补零，但这里没写)
        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len)
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # 将说话人ID转为long类型以便计算loss
        speaker = torch.FloatTensor([speaker]).long()
        # 返回 (mel_spectrogram, speaker_id)
        return mel, speaker
    
    # 有多少个不同的说话人
    def get_speaker_number(self):
        return self.speaker_num

print("Complete")

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

print("Complete")

# Transformer
## 一种基于 自注意力机制（Self-Attention） 的深度学习模型架构
## Transformer 的核心是 自注意力机制（Self-Attention），它可以让模型在处理序列数据（如文本）时，动态地关注输入的不同部分，而不像 RNN/LSTM 那样依赖固定的顺序计算。
### 输入：一个序列（如句子中的单词）
### 计算过程：Query, Key, Value（QKV）：每个输入单词会被转换成 3 个向量：Query（查询）：表示当前单词要查询其他单词的重要性。Key（键）：表示当前单词对其他单词的贡献程度。Value（值）：表示当前单词的实际信息。计算注意力分数：计算 Query 和 Key 的点积，得到 注意力分数（表示单词之间的相关性）。用 Softmax 归一化，得到 注意力权重。加权求和：用注意力权重对 Value 加权求和，得到最终的 注意力输出。
## Transformer 由 编码器（Encoder） 和 解码器（Decoder） 组成
## Transformer 的关键参数
### d_model,输入/输出的特征维度（如词向量维度）：通常取 512、768、1024，取决于任务复杂度。更大的 d_model 能存储更多信息，但计算量也更大。
### nhead,多头注意力的头数（并行计算）:一般取 8~16，需能被 d_model 整除（如 d_model=512, nhead=8 → 每个头维度=512/8=64）
### num_layers,Encoder/Decoder 的层数:num_layers：简单任务 6 层，大模型（如 GPT-3）可达 96 层。
### dim_feedforward	前馈神经网络的隐藏层维度:通常取 4 × d_model（如 d_model=512 → dim_feedforward=2048）。
## 比 RNN/LSTM 更高效、更强大，适用于多种任务。如果你要实现一个 Transformer，可以使用 PyTorch 的 nn.Transformer 模块，或直接调用 Hugging Face 的现成模型（如 BERT、GPT-2）。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
	def __init__(self, d_model=80, n_spks=600, dropout=0.1):
		super().__init__()

        # 特征预处理 (prenet): 将输入的特征维度转换为模型内部所需的维度。
		self.prenet = nn.Linear(40, d_model)

        # 序列编码 (encoder_layer / encoder): 使用 Transformer 编码器来捕捉输入序列（语音帧）中的时序依赖和上下文信息。
		self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=256, nhead=2
		)
		self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # 最终预测 (pred_layer): 将编码器输出的序列信息聚合成固定表示，然后映射到最终的说话人分类得分。 
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.Sigmoid(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# 将原始的语音特征调整到了你的 Transformer 编码器期望的特征维度 (d_model)
		out = self.prenet(mels)
		# 交换张量的维度
		out = out.permute(1, 0, 2)
		# 对输入序列的深度特征提取,每经过一层，模型对序列的理解就更深入一些
		out = self.encoder_layer(out)
		# 交换张量 out 的前两个维度（索引为 0 和 1 的维度）和上面那个不一样,要区分
		out = out.transpose(0, 1)
		# 算张量 out 沿着第二个维度（索引为 1 的维度）的平均值
		stats = out.mean(dim=1)

		# 输入传递给模型中的一个预测层 
		out = self.pred_layer(stats)
		return out

print("Complete")

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer, # 要调度的优化器
	num_warmup_steps: int, # 预热步数
	num_training_steps: int, # 总训练步数
	num_cycles: float = 0.5, # 余弦周期数
	last_epoch: int = -1, # 恢复训练时指定上次的epoch
):
	
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

In [ ]:
import torch


def model_fn(batch, model, criterion, device):

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)
    
    # 将输入数据传递给模型，并获取模型的输出。
	outs = model(mels)
    # 损失函数，计算loss代表了模型当前表现得有多‘差’
    # loss 的值越小，表示模型的预测越接近真实情况，模型表现得越好。
    # loss 的值越大，表示模型的预测与真实情况偏差越大，模型表现得越差。
	loss = criterion(outs, labels)

	# 对于每个输入样本，把该类别的索引作为最终的预测结果
	preds = outs.argmax(1)
    # 计算模型预测正确的样本占总样本数的比例
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 

    # 切换到评估模式
	model.eval()
    
	running_loss = 0.0
	running_accuracy = 0.0

    # 创建进度条
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()
        # 更新进度条的显示
		pbar.update(dataloader.batch_size)
        # 进度条的右侧实时显示额外信息
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)
    #关闭并清理进度条
	pbar.close()
    #切换到训练模式
	model.train()
    
    # 计算并返回模型在整个验证集上的平均准确率
	return running_accuracy / len(dataloader)

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
# 训练像 Transformer 这样的大型模型时，AdamW 通常是更推荐的选择。带解耦权重衰减的 Adam。
# 这是因为模型的参数量巨大，L2 正则化（权重衰减）对于防止过拟合至关重要。
from torch.utils.data import DataLoader, random_split


def parse_args():

	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 256,
		"n_workers": 16,
		"valid_steps": 1000,
		"warmup_steps": 2000,
		"save_steps": 5000,
		"total_steps": 20000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):

	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
    # 存储模型参数（权重和偏置）的副本
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):

		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

	
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)


		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)


			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")


		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

In [ ]:
import os # 提供了与操作系统交互的功能，例如路径拼接
import json # 用于处理 JSON 格式的数据
import torch # 这是深度学习框架，用于张量操作等
from pathlib import Path # 从 pathlib 模块导入 Path 类，它提供了面向对象的文件系统路径操作，比 os.path 更现代和方便。
from torch.utils.data import Dataset #  从 PyTorch 的 utils.data 模块导入 Dataset 基类。所有自定义的数据集类都需要继承它。


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		# Path(data_dir) 将 data_dir 字符串转换为一个 Path 对象，然后 / 运算符用于路径拼接，这比 os.path.join 更直观和安全。
        testdata_path = Path(data_dir) / "testdata.json" 
		# 打开 testdata_path 指向的 JSON 文件（testdata_path.open() 返回一个文件对象），然后使用 json.load() 函数加载并解析 JSON 数据到 metadata 字典中。
        metadata = json.load(testdata_path.open())
		# 特征文件的根目录
        self.data_dir = data_dir
        # 从加载的 metadata 字典中，提取键为 "utterances" 的值，并将其存储为实例的属性 self.data。
        # "utterances" 应该是一个列表，其中每个元素代表一个要进行推理的语音片段（或称为“话语”），并包含其特征文件的路径等信息。
		self.data = metadata["utterances"]

	def __len__(self):
        # 在数据集对象上调用 len() 函数时（例如 len(dataset_instance)），这个方法会被调用。
		return len(self.data)

	def __getitem__(self, index):
        # 这是 Dataset 类另一个必需的方法。它使得数据集对象可以像列表一样通过索引来访问（例如 dataset_instance[0]）。当给定一个 index 时，这个方法应该返回对应索引的数据样本。
		utterance = self.data[index]
        # 从当前 utterance 字典中提取键为 "feature_path" 的值。这应该是相对于 self.data_dir 的特征文件的路径。
		feat_path = utterance["feature_path"]
        # 指定的路径加载一个 PyTorch 张量
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel

# 这个函数用于将多个单独的数据样本（由 __getitem__ 返回）组合成一个批次（batch）
def inference_collate_batch(batch):
	# feat_paths 将是一个包含所有文件路径的元组（(path1, path2, ...)），mels 将是一个包含所有梅尔频谱图张量的元组（(mel1, mel2, ...)）。
	# *batch: 将 batch 列表解包，例如如果 batch 是 [(path1, mel1), (path2, mel2)]，那么 *batch 就会变成 (path1, mel1), (path2, mel2)。
    # zip(...): zip 函数将这些独立的元组重新组合。它会把所有第一个元素打包成一个元组，所有第二个元素打包成另一个元组。
    feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

In [ ]:
import json
import csv # 用于写入 CSV 文件（保存推理结果）
from pathlib import Path
from tqdm.notebook import tqdm # 从tqdm.notebook导入 tqdm，这是一个进度条库，用于在循环中显示进度，特别适合 Jupyter Notebook 环境。

import torch
from torch.utils.data import DataLoader

def parse_args():
	# 配置参数
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
    # 语法糖
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
        #  使用 8 个子进程来并行加载数据。这可以加速数据加载过程。
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
        # PyTorch 将不会计算梯度。这对于推理阶段非常重要，因为推理不需要反向传播来更新权重，禁用梯度计算可以节省内存并加速推理。
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

    # 以写入模式 ('w') 打开指定的 output_path 文件。newline='' 对于 CSV 文件很重要，可以防止写入额外的空行。
	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)

# 这是一个标准的 Python 惯用法，确保 main() 函数只在脚本作为主程序直接运行时才执行，而不是在被其他模块导入时执行。
if __name__ == "__main__":
	main(**parse_args())
    # 调用 main 函数。**parse_args() 会解包 parse_args() 返回的字典，将其中的键值对作为关键字参数传递给 main 函数，
    # 例如 main(data_dir="./Dataset", model_path="./model.ckpt", output_path="./output.csv")。

# 本来想选择openai/whisper-large-v3再优化一下跑一跑,但是显存不够没跑起来

In [ ]:
!pip install -U transformers

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")

In [ ]:
import os
import json
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperFeatureExtractor, WhisperForAudioClassification
from torch.optim import AdamW
from tqdm import tqdm

# 1. 数据加载
class SpeakerDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        with open(f"{data_dir}/mapping.json") as f:
            self.speaker2id = json.load(f)["speaker2id"]
        with open(f"{data_dir}/metadata.json") as f:
            metadata = json.load(f)
        self.data = [
            (utt["feature_path"], self.speaker2id[spk]) 
            for spk in metadata["speakers"] 
            for utt in metadata["speakers"][spk]
        ]
        self.feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, speaker = self.data[idx]
        mel = torch.load(f"{self.data_dir}/{path}").numpy()
        inputs = self.feature_extractor(mel, sampling_rate=16000, return_tensors="pt")
        return inputs.input_features[0], torch.tensor(speaker)

# 2. 数据处理
def collate_fn(batch):
    features, labels = zip(*batch)
    features = torch.stack(features)
    return features, torch.stack(labels)

# 3. 模型定义
class SpeakerClassifier(torch.nn.Module):
    def __init__(self, num_speakers):
        super().__init__()
        self.whisper = WhisperForAudioClassification.from_pretrained(
            "openai/whisper-large-v3",
            num_labels=num_speakers
        )
        # 冻结大部分层
        for param in self.whisper.parameters():
            param.requires_grad = False
        # 解冻分类层
        for param in self.whisper.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.whisper(x).logits

# 4. 训练函数
def train(data_dir, epochs=5, batch_size=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 加载数据
    dataset = SpeakerDataset(data_dir)
    train_size = int(0.9 * len(dataset))
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset)-train_size])
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_set, batch_size=batch_size, collate_fn=collate_fn)

    # 初始化模型
    model = SpeakerClassifier(len(dataset.speaker2id)).to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # 训练循环
    for epoch in range(epochs):
        model.train()
        for features, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            features, labels = features.to(device), labels.to(device)
            
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 验证
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f"验证准确率: {correct/total:.4f}")

    return model

# 5. 预测函数
def predict(model, audio_features):
    device = next(model.parameters()).device
    model.eval()
    with torch.no_grad():
        inputs = model.feature_extractor(audio_features, return_tensors="pt").input_features.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# 使用示例
if __name__ == "__main__":
    # 训练模型
    model = train("Dataset")
    
    # 保存模型
    torch.save(model.state_dict(), "speaker_classifier.pt")
    
    # 加载测试数据示例
    test_mel = torch.load("Dataset/features/test_sample.pt")  # 替换为你的测试数据
    predicted_id = predict(model, test_mel.numpy())
    print(f"预测说话人ID: {predicted_id}")